<a href="https://colab.research.google.com/github/manushree635/CV/blob/main/unetsegmentation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries


In [ ]:
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms.functional as F
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
from distutils.file_util import copy_file


UNet Implementation
 


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)
    
    

In [ ]:
class UNet(nn.Module):
    def __init__(self,in_channels=3,out_channels=1,channels=[64,128,256,512]):
        super(UNet,self).__init__()
        self.downs=nn.ModuleList()
        self.ups=nn.ModuleList()
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)


        for channel in channels:
            self.downs.append(DoubleConv(in_channels,channel))
            in_channels=channel

        for channel in reversed(channels):
            self.ups.append(nn.ConvTranspose2d(channel*2,channel,kernel_size=2,stride=2))
            self.ups.append(DoubleConv(channel*2,channel))
    
        self.bottleneck=DoubleConv(channels[-1],channels[-1]*2)
        self.final=nn.Conv2d(channels[0],out_channels,kernel_size=1)
        
     
    
    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip_connection = skip_connections[i//2]

            if x.shape != skip_connection.shape:
                x = F.resize(x, size=skip_connection.shape[2:])

            concat = torch.cat((skip_connection, x), dim=1)
            x = self.ups[i+1](concat)

        return self.final(x)


Splitting Dataset into Train and Valid Sets


In [ ]:

trainPath = '/content/train'
maskPath = '/content/train_masks'

In [ ]:
listt = []
listm = []



for root, directories, files in os.walk(trainPath):
    for name in files:
        listt.append(os.path.join(root, name))
        

for root, directories, files in os.walk(maskPath):
    for name in files:
        listm.append(os.path.join(root, name))

listt.sort()
listm.sort()



In [ ]:
!mkdir ./Data

# Train data folders
!mkdir ./Data/Train/
!mkdir ./Data/Train/Images/
!mkdir ./Data/Train/Masks/


# Validation data folders
!mkdir ./Data/Val/
!mkdir ./Data/Val/Images/
!mkdir ./Data/Val/Masks/

In [ ]:
pathImagesTrain = './Data/Train/Images/'
pathImagesValid = './Data/Val/Images/'


pathMasksTrain = './Data/Train/Masks/'
pathMasksValid = './Data/Val/Masks/'

In [ ]:
for i in range(len(listt)):
    if i < 48:
        copy_file(listt[i],pathImagesValid )
        copy_file(listm[i],pathMasksValid )

    else:
        copy_file(listt[i],pathImagesTrain )
        copy_file(listm[i],pathMasksTrain )

In [ ]:
class CarvanaDataset(Dataset):
  def __init__(self,image_dir,mask_dir,transform=None):
    self.image_dir=image_dir
    self.mask_dir=mask_dir
    self.transform=transform
    self.images=os.listdir(image_dir)
  
  def __len__(self):
    return len(self.images)

  def __getitem__(self,index):
    image_path=os.path.join(self.image_dir,self.images[index])
    mask_path=os.path.join(self.mask_dir,self.images[index].replace('.jpg','_mask.gif'))
    image=np.array(Image.open(image_path).convert("RGB"))
    mask=np.array(Image.open(mask_path).convert('L'),dtype=np.float32)
    mask[mask==255.0]=1.0
    if self.transform is not None:
      aug=self.transform(image=image,mask=mask)
      image=aug['image']
      mask=aug['mask']
    
      
    return image,mask




Loading Dataset

In [ ]:
# Hyperparameters
LEARNING_RATE = 1e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16
num_epochs = 7

IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally

LOAD_MODEL = False
train_img_dir = "/content/Data/Train/Images"
train_mask_dir = "/content/Data/Train/Masks"
val_img_dir = "/content/Data/Val/Images"
val_mask_dir="/content/Data/Val/Masks"



train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

val_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

model = UNet(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


train_ds = CarvanaDataset(
        image_dir=train_img_dir,
        mask_dir=train_mask_dir,
        transform=train_transform,
    )

train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
    )

val_ds = CarvanaDataset(
        image_dir=val_img_dir,
        mask_dir=val_mask_dir,
        transform=val_transform,
    )

val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
    )


Training and Validating the Model

In [ ]:


def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="/content/Data/saved_images", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

In [ ]:
import tqdm.notebook as tq
check_accuracy(val_loader, model, device=DEVICE)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    loop = tq.tqdm(train_loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

        # check accuracy
    check_accuracy(val_loader, model, device=DEVICE)

        # saving  predictions
    save_predictions_as_imgs(
            val_loader, model, folder="/content/Data/saved_images", device=DEVICE
        )

Got 1466253/1843200 with acc 79.55
Dice score: 0.23532848060131073



Got 1735052/1843200 with acc 94.13
Dice score: 0.8805936574935913



Got 1767626/1843200 with acc 95.90
Dice score: 0.9135105013847351



Got 1771724/1843200 with acc 96.12
Dice score: 0.919445276260376



Got 1796250/1843200 with acc 97.45
Dice score: 0.9441395998001099



Got 1775387/1843200 with acc 96.32
Dice score: 0.9234609603881836



Got 1798829/1843200 with acc 97.59
Dice score: 0.9485578536987305



Got 1802985/1843200 with acc 97.82
Dice score: 0.9529500007629395
